In [2]:
from helpers import data, quantify

In [3]:
data?

In [4]:
passports = data(4, sep="\n\n")
passports[0]

'byr:1971\necl:hzl pid:112040163\neyr:2023 iyr:2019\nhcl:#b6652a hgt:167cm'

In [5]:
passports = list(map(lambda s: s.replace("\n", " "), passports))
passports[0]

'byr:1971 ecl:hzl pid:112040163 eyr:2023 iyr:2019 hcl:#b6652a hgt:167cm'

In [6]:
passports = list(map(lambda s: s.split(), passports))
passports[0]

['byr:1971',
 'ecl:hzl',
 'pid:112040163',
 'eyr:2023',
 'iyr:2019',
 'hcl:#b6652a',
 'hgt:167cm']

In [7]:
passports = list(map(lambda arr: dict((s[:3],  s[4:]) for s in arr) , passports))
passports[0]

{'byr': '1971',
 'ecl': 'hzl',
 'pid': '112040163',
 'eyr': '2023',
 'iyr': '2019',
 'hcl': '#b6652a',
 'hgt': '167cm'}

**Part 1:** Report number of passports with fields byr, iyr, eyr, hgt, hcl, ecl, pid (don't care about cid). 

In [8]:
required = { "byr", "iyr", "eyr", "hgt", "hcl", "ecl", "pid" }

In [9]:
def attempt_one():
    valid = 0 
    for d in passports: 
        fields = set(d.keys())
        valid += required.issubset(fields)
    return valid 

attempt_one()

192

In [10]:
def attempt_two():
    valid = 0 
    for d in passports: 
        fields = set(d.keys())
        valid += fields | required == fields # OR is same as original 
    return valid

attempt_two()

192

In [11]:
# Dammit Norvig 
import re
def parse_passport(passport: str) -> dict:
    "Make a dict with all key:value entries in passport"
    return dict(re.findall(r"([a-z]+):([^\s]+)", passport))

assert parse_passport("one:h3 to:12_\nfour:4") == {"one": "h3", "to": "12_", "four": "4"}

In [12]:
def attempt_three():
    # Norvig 
    passports = data(4, parser=parse_passport, sep="\n\n")
    return quantify(passports, required.issubset) # Wowza 

attempt_three()

192

**Part 2:** Validate field values now. 

In [37]:
in_between = lambda a, b: lambda x: a <= int(x) <= b

def hgt(s):
    if s[-2:] == "cm": 
        return in_between(150, 193)(s[:-2])
    elif s[-2:] == "in":
        return in_between(59, 76)(s[:-2])
    else: 
        return False
#         raise ValueError(f"yo what it was {s[-2:]}, full s is {s}")
        
def hcl(s):
    if s[0] != "#": 
        return False 
    if re.fullmatch("[0-9a-f]{6}$", s[1:]):
        return True 
    return False 


checks = {
    "byr": in_between(1920, 2002), 
    "iyr": in_between(2010, 2020), 
    "eyr": in_between(2020, 2030), 
    "hgt": hgt, 
    "hcl": hcl, 
    "ecl": lambda s: s in {"amb", "blu", "brn", "gry", "grn", "hzl", "oth"}, 
    "pid": lambda s: re.fullmatch("[0-9]{9}#", s)
}

# Cleaner method 
checks = dict(
    byr=in_between(1920, 2002), 
    iyr=in_between(2010, 2020), 
    eyr=in_between(2020, 2030), 
    hgt=hgt, 
    hcl=hcl, 
    ecl=lambda s: s in {"amb", "blu", "brn", "gry", "grn", "hzl", "oth"}, 
    pid=lambda s: re.fullmatch("[0-9]{9}$", s)
)

In [44]:
def attempt_one():
    valid = 0 
    for d in passports: 
        fields = set(d.keys())
        # Required should have all fields
        if fields | required != fields: 
            continue 
        for field, func in checks.items():
            if not func(d[field]):
                break # Finally get to use for/else !
        else: 
            valid += 1 
    return valid 

attempt_one()

101

In [41]:
# Norvig 
def attempt_two():
    return sum(all(field in passport and checks[field](passport[field]) 
                  for field in required)
               for passport in passports
              )
attempt_two()

101

In [47]:
# Compare 
from timeit import timeit 
print(timeit(attempt_one, number=1000))
print(timeit(attempt_two, number=1000))

0.65501359100017
0.6731009589998393
